In [1]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil
import logging
logger = logging.getLogger(str(os.getpid()))

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, timelag_predictors, timelag_predictors_target, roll_predictors_target
from wattile.time_processing import add_processed_time_columns
PROJECT_DIRECTORY = Path().resolve().parent.parent

# reading configs

In [2]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# main configs file
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)
##################################################################################
# code testing configs file
# with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_input"]["exp_dir"] = str(PROJECT_DIRECTORY / exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "data" / "Synthetic Site")

configs

{'data_input': {'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\data\\Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power',
  'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir'},
 'data_output': {'exp_dir': 'exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'fea

# reading data

In [3]:
configs["target_feat_name"] = [configs["data_input"]["target_var"]]
data = read_dataset_from_file(configs)
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


# data processing

In [4]:
# assert we have the correct columns and order them
data = correct_predictor_columns(configs, data)

# sort and trim data specified time period
data = correct_timestamps(configs, data)

# Add time-based features
data = add_processed_time_columns(data, configs)

# Add statistics features
data = resample_or_rolling_stats(data, configs)

data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,HOD_binary_reg_0_min,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,103.650002
2021-12-01 07:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:45:00+00:00,16.614972,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.349998
2021-12-01 08:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,0.0,0.0,0.705556,0.294444,0.0,0.0,0.0,-0.493776,0.869589,102.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.000000,0.998135,-1.608123e-16,0.0,...,0.0,0.0,0.788889,0.211111,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.000000,0.997859,-6.540313e-02,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,102.709602
2021-12-08 06:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.000000,0.991445,-1.305262e-01,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,105.972198


In [5]:
# configs["learning_algorithm"]["arch_version"] = "charlie"

In [6]:
# if configs["learning_algorithm"]["arch_version"] == "alfa":
#     data = timelag_predictors(data, configs)
# elif configs["learning_algorithm"]["arch_version"] == "bravo":
#     data = timelag_predictors_target(data, configs)
# elif configs["learning_algorithm"]["arch_version"] == "charlie":
#     data = roll_predictors_target(data, configs)
    
# data.to_csv("./output.csv")

# data

In [7]:
# data_temp = data.loc[:, 
#          (data.columns.str.contains("Synthetic Weather Station Dew Point Temperature_mean")) |
#          data.columns.str.contains("Synthetic Site Electricity Main Total Power")
#         ]

# data_temp.to_csv("./output.csv")

# data triming testing

In [37]:
configs["learning_algorithm"]["arch_version"] = "bravo"
configs["data_processing"]["input_output_window"]["window_width_futurecast"] = "30min"

In [38]:
timestamp_cast = pd.to_datetime("2021-12-03 13:30:00+00:00")
timestamp_cast

Timestamp('2021-12-03 13:30:00+0000', tz='UTC')

In [39]:
config_data_processing = configs["data_processing"]
config_feat_timelag = config_data_processing["feat_timelag"]
config_input_output_window = config_data_processing["input_output_window"]

if configs["learning_algorithm"]["arch_version"] == "alfa":
    
    window_start_offset = pd.Timedelta(config_feat_timelag["lag_interval"]) \
    * config_feat_timelag["lag_count"]

    window_end_offset = pd.Timedelta("0min")
    
elif configs["learning_algorithm"]["arch_version"] == "bravo": 
    
    window_start_offset = pd.Timedelta(config_feat_timelag["lag_interval"]) \
    * config_feat_timelag["lag_count"]

    window_end_offset = pd.Timedelta(config_input_output_window["window_width_target"]) \
    - pd.Timedelta(config_data_processing["resample_interval"])
    
    timestamp_cast = timestamp_cast - window_end_offset
    
elif configs["learning_algorithm"]["arch_version"] == "charlie":
    
    window_start_offset = pd.Timedelta(config_input_output_window["window_width_source"])
    
    window_end_offset = pd.Timedelta(config_input_output_window["window_width_target"]) \
    - pd.Timedelta(config_data_processing["resample_interval"])

print("window_start_offset = {}".format(window_start_offset))
print("window_end_offset = {}".format(window_end_offset))

prediction_window_start_time = timestamp_cast - window_start_offset
prediction_window_end_time = timestamp_cast + window_end_offset 

print("window_start_time = {}".format(prediction_window_start_time))
print("window_end_time = {}".format(prediction_window_end_time))

configs["data_input"]["prediction_window_start_time"] = prediction_window_start_time
configs["data_input"]["prediction_window_end_time"] = prediction_window_end_time 

window_start_offset = 0 days 06:00:00
window_end_offset = 0 days 00:30:00
window_start_time = 2021-12-03 07:00:00+00:00
window_end_time = 2021-12-03 13:30:00+00:00


In [40]:
data_test = data.loc[prediction_window_start_time:prediction_window_end_time, :]
data_test

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,HOD_binary_reg_0_min,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-03 07:00:00+00:00,17.936412,-1.565144,-0.805857,59.360001,-1.954443,18.129999,8.585611,9.659258e-01,-0.258819,0.0,...,0.0,0.0,0.0,0.0,0.747222,0.252778,0.0,-0.46355,0.886071,103.349998
2021-12-03 07:15:00+00:00,17.856106,-1.598714,-1.034617,59.180000,-1.936150,17.780001,8.332830,9.469301e-01,-0.321439,0.0,...,0.0,0.0,0.0,0.0,0.708333,0.291667,0.0,-0.46355,0.886071,96.599998
2021-12-03 07:30:00+00:00,17.689230,-1.601668,-1.148990,58.675999,-1.968473,17.590000,8.780231,9.238795e-01,-0.382683,0.0,...,0.0,0.0,0.0,0.0,0.708333,0.291667,0.0,-0.46355,0.886071,98.000000
2021-12-03 07:45:00+00:00,17.547188,-1.497059,-0.613491,58.189999,-1.976861,18.549999,8.417836,8.968727e-01,-0.442289,0.0,...,0.0,0.0,0.0,0.0,0.708333,0.291667,0.0,-0.46355,0.886071,103.250000
2021-12-03 08:00:00+00:00,17.375589,-1.572855,-0.878647,57.452000,-2.022473,18.610001,7.411185,8.660254e-01,-0.500000,0.0,...,0.0,0.0,0.0,0.0,0.705556,0.294444,0.0,-0.46355,0.886071,96.550003
2021-12-03 08:15:00+00:00,17.184826,-1.723750,-1.653318,58.279999,-1.953886,17.709999,8.053205,8.314696e-01,-0.555570,0.0,...,0.0,0.0,0.0,0.0,0.666667,0.333333,0.0,-0.46355,0.886071,98.400002
2021-12-03 08:30:00+00:00,17.244626,-1.578048,-0.951425,57.722000,-1.975832,18.059999,8.276905,7.933533e-01,-0.608761,0.0,...,0.0,0.0,0.0,0.0,0.666667,0.333333,0.0,-0.46355,0.886071,97.050003
2021-12-03 08:45:00+00:00,17.302342,-1.637828,-1.299766,58.891998,-1.871101,17.670000,9.954656,7.518398e-01,-0.659346,0.0,...,0.0,0.0,0.0,0.0,0.666667,0.333333,0.0,-0.46355,0.886071,103.400002
2021-12-03 09:00:00+00:00,17.484329,-1.632004,-0.863036,59.054001,-2.002500,17.770000,7.885430,7.071068e-01,-0.707107,0.0,...,0.0,0.0,0.0,0.0,0.663889,0.336111,0.0,-0.46355,0.886071,103.000000


In [41]:
if configs["learning_algorithm"]["arch_version"] == "alfa":
    data_test = timelag_predictors(data_test, configs)
    print("ALFA")
elif configs["learning_algorithm"]["arch_version"] == "bravo":
    data_test = timelag_predictors_target(data_test, configs)
    print("BRAVO")
elif configs["learning_algorithm"]["arch_version"] == "charlie":
    data_test = roll_predictors_target(data_test, configs)
    print("CHARLIE")
    
data_test

BRAVO


C:\Users\JKIM4\Anaconda3\envs\wattile\lib\site-packages\wattile\buildings_processing.py:246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[target_var] = data[target_var].shift(freq="-" + window_width_futurecast)


,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power_lag_0,Synthetic Site Electricity Main Total Power_lag_1,Synthetic Site Electricity Main Total Power_lag_2
Timestamp,,,,,,,,,,,,,,,,,,,,,


In [21]:
data_test

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-03 13:30:00+00:00,17.68923,-1.601668,-1.14899,58.675999,-1.968473,17.59,8.780231,0.92388,-0.382683,0.0,...,0.0,0.0,0.0,0.0,0.458333,0.541667,0.0,-0.46355,0.886071,112.783997
